**Diana Kim ADA-2403M**

# **Практическая работа №5**
**Тема:** Реализация параллельных структур данных на GPU.

**Цель работы:**
1. Освоить программирование параллельных структур данных с
использованием CUDA.
2. Реализовать параллельные структуры данных (например, параллельный
стек и очередь).
3. Провести исследование производительности реализованных структур
данных.

In [1]:
!nvidia-smi

Fri Jan 16 11:03:04 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


***Лабораторная работа разделена на 3 части***

### **1 часть. Параллельный стек LIFO**

Здесь мне необходимо реализовать параллельный стек LIFO на GPU с использованием глобальной памяти и атомарных операций, обеспечив корректное выполнение операций добавления и извлечения элементов при одновременной работе нескольких потоков.





In [8]:
%%writefile part1.cu

#include <cuda_runtime.h>        // для функций CUDA, которые мне надо
#include <iostream>                  // ввод/вывод
#include <cstdlib>                             // подключает exit()
using namespace std;                           // чтобы не писать std



void cuda_ok(cudaError_t err, const char* msg) {              // функция проверки ошибок CUDA
    if (err != cudaSuccess) {                         // если произошла ошибка
        cout << "CUDA error (" << msg << "): "           // печатает место ошибки
             << cudaGetErrorString(err) << endl;              // печатает текст ошибки
        exit(1);                                           // завершает программу
    }
}


struct Stack {                    // структура стека
    int* data;                // массив данных стека в GPU памяти
    int top;                           // индекс вершины
    int capacity;                // максимальная емкость стека

    __device__ void init(int* buffer, int size) {                     // инициализация стека на GPU
        data = buffer;                                     // запоминает указатель на буфер
        top = -1;                                      // делает стек пустым
        capacity = size;                                 // сохраняет ёмкость
    }

    __device__ bool push(int value) {              // кладёт элемент в стек
        int pos = atomicAdd(&top, 1) + 1;                 // атомарно увеличивает top и получает позицию
        if (pos < capacity) {                  // если место есть
            data[pos] = value;                      // записывает значение в стек
            return true;                     // сообщает успех
        }
        atomicSub(&top, 1);                         // откатывает top назад, если места нет
        return false;                  // сообщает неуспех
    }

    __device__ bool pop(int* value) {                  // достаёт элемент из стека
        int pos = atomicSub(&top, 1);                      // атомарно берёт текущий top и уменьшает его
        if (pos >= 0) {                        // если стек не был пустым
            *value = data[pos];                          // записывает извлечённое значение
            return true;                 // сообщает успех
        }
        atomicAdd(&top, 1);                              // откатывает top назад, если был пустой
        return false;                             // сообщает неуспех
    }
};




__global__ void kernel_init(Stack* st, int* buffer, int cap) {                   // ядро для инициализации стека
    if (threadIdx.x == 0 && blockIdx.x == 0) {                 // только один поток выполняет
        st->init(buffer, cap);                        // инициализирует стек
    }
}

__global__ void kernel_push(Stack* st, int n_push, int* ok_push) {                // ядро для параллельных push
    int tid = threadIdx.x + blockIdx.x * blockDim.x;                         // вычисляет глобальный id потока
    if (tid < n_push) {                                           // если поток входит в число push
        bool ok = st->push(tid);                             // кладёт tid в стек
        ok_push[tid] = ok ? 1 : 0;                             // сохраняет 1 если push успешен
    }
}

__global__ void kernel_pop(Stack* st, int n_pop, int* out, int* ok_pop) {                    // ядро для параллельных pop
    int tid = threadIdx.x + blockIdx.x * blockDim.x;                                 // вычисляет глобальный id потока
    if (tid < n_pop) {                                                     // если поток входит в число pop
        int val = -1;                                                  // создаёт переменную для значения
        bool ok = st->pop(&val);                               // пытается достать значение из стека
        out[tid] = val;                              // записывает извлечённое значение
        ok_pop[tid] = ok ? 1 : 0;                        // сохраняет 1 если pop успешен
    }
}

__global__ void kernel_get_top(Stack* st, int* out_top) {                   // ядро чтобы прочитать top
    if (threadIdx.x == 0 && blockIdx.x == 0) {                        // только один поток выполняет
        out_top[0] = st->top;                          // копирует top в массив
    }
}





int main() {
    const int CAP = 1024;                     // емкость стека
    const int N_PUSH = 512;                      // сколько элементов кладем
    const int N_POP  = 512;                        // сколько элементов достаем
    int* d_buffer = nullptr;               // буфер данных стека на GPU
    Stack* d_stack = nullptr;                         // объект стека на GPU
    int* d_ok_push = nullptr;                  // массив успешности push на GPU
    int* d_ok_pop  = nullptr;                   // массив извлечённых значений на GPU
    int* d_top_val = nullptr;              // массив для top на GPU
    int* d_out = nullptr;          // массив извлечённых значений на GPU

    cuda_ok(cudaMalloc((void**)&d_buffer, CAP * (int)sizeof(int)), "cudaMalloc d_buffer");                // выделяет буфер стека
    cuda_ok(cudaMalloc((void**)&d_stack, (int)sizeof(Stack)), "cudaMalloc d_stack");                       // выделяет память под Stack
    cuda_ok(cudaMalloc((void**)&d_ok_push, N_PUSH * (int)sizeof(int)), "cudaMalloc d_ok_push");                   // выделяет ok_push
    cuda_ok(cudaMalloc((void**)&d_ok_pop,  N_POP  * (int)sizeof(int)), "cudaMalloc d_ok_pop");                    // выделяет ok_pop
    cuda_ok(cudaMalloc((void**)&d_out,     N_POP  * (int)sizeof(int)), "cudaMalloc d_out");                  // выделяет out
    cuda_ok(cudaMalloc((void**)&d_top_val, (int)sizeof(int)), "cudaMalloc d_top_val");                          // выделяет top_val

    kernel_init<<<1, 1>>>(d_stack, d_buffer, CAP);                     // запускает инициализацию стека
    cuda_ok(cudaGetLastError(), "kernel_init launch");                    // проверяет запуск
    cuda_ok(cudaDeviceSynchronize(), "kernel_init sync");                 // ждет завершения

    int blockSize = 256;                                       // размер блока
    int gridPush = (N_PUSH + blockSize - 1) / blockSize;                       // количество блоков для push
    int gridPop  = (N_POP  + blockSize - 1) / blockSize;                        // количество блоков для pop

    kernel_push<<<gridPush, blockSize>>>(d_stack, N_PUSH, d_ok_push);              // параллельно кладет значения
    cuda_ok(cudaGetLastError(), "kernel_push launch");                      // проверяет запуск
    cuda_ok(cudaDeviceSynchronize(), "kernel_push sync");                      // ждет завершения
    kernel_pop<<<gridPop, blockSize>>>(d_stack, N_POP, d_out, d_ok_pop);                  // параллельно достает значения
    cuda_ok(cudaGetLastError(), "kernel_pop launch");                      // проверяет запуск
    cuda_ok(cudaDeviceSynchronize(), "kernel_pop sync");                            // ждет завершения
    kernel_get_top<<<1, 1>>>(d_stack, d_top_val);                            // читает финальный top
    cuda_ok(cudaGetLastError(), "kernel_get_top launch");                        // проверяет запуск
    cuda_ok(cudaDeviceSynchronize(), "kernel_get_top sync");                       // ждет завершения

    int* h_ok_push = new int[N_PUSH];                         // массив ok_push на CPU
    int* h_ok_pop  = new int[N_POP];                                  // массив ok_pop на CPU
    int* h_out     = new int[N_POP];                            // массив out на CPU
    int  h_top     = 0;                               // переменная top на CPU
    cuda_ok(cudaMemcpy(h_ok_push, d_ok_push, N_PUSH * (int)sizeof(int), cudaMemcpyDeviceToHost), "copy ok_push");                 // копирует ok_push
    cuda_ok(cudaMemcpy(h_ok_pop,  d_ok_pop,  N_POP  * (int)sizeof(int), cudaMemcpyDeviceToHost), "copy ok_pop");                    // копирует ok_pop
    cuda_ok(cudaMemcpy(h_out,     d_out,     N_POP  * (int)sizeof(int), cudaMemcpyDeviceToHost), "copy out");                  // копирует out
    cuda_ok(cudaMemcpy(&h_top,    d_top_val, (int)sizeof(int),           cudaMemcpyDeviceToHost), "copy top");                      // копирует top

    int push_success = 0;                         // счетчик успешных push
    for (int i = 0; i < N_PUSH; i++) {                   // цикл по push
        push_success += h_ok_push[i];                       // суммирует успехи push
    }
    int pop_success = 0;                              // счетчик успешных pop
    for (int i = 0; i < N_POP; i++) {                       // цикл по pop
        pop_success += h_ok_pop[i];                    // суммирует успехи pop
    }

    cout << "Push success: " << push_success << " / " << N_PUSH << endl;              // печатает сколько push прошло
    cout << "Pop  success: " << pop_success  << " / " << N_POP  << endl;                // печатает сколько pop прошло
    cout << "Final top value: " << h_top << endl;              // печатает финальный top
    cout << "10 popped values: ";
    for (int i = 0; i < 10 && i < N_POP; i++) {                 // выводит первые 10 значений
        cout << h_out[i] << " ";
    }
    cout << endl;

    cuda_ok(cudaFree(d_buffer), "cudaFree d_buffer");                // освобождает буфер стека
    cuda_ok(cudaFree(d_stack), "cudaFree d_stack");                      // освобождает Stack
    cuda_ok(cudaFree(d_ok_push), "cudaFree d_ok_push");                // освобождает ok_push
    cuda_ok(cudaFree(d_ok_pop), "cudaFree d_ok_pop");                    // освобождает ok_pop
    cuda_ok(cudaFree(d_out), "cudaFree d_out");                        // освобождает out
    cuda_ok(cudaFree(d_top_val), "cudaFree d_top_val");                   // освобождает top_val

    delete[] h_ok_push;                             // освобождает ok_push на CPU
    delete[] h_ok_pop;                                 // освобождает ok_pop на CPU
    delete[] h_out;                               // освобождает out на CPU

    return 0;                  // завершает программу
}

Overwriting part1.cu


In [9]:
!nvcc part1.cu -o part1 -arch=compute_75 -code=sm_75

In [10]:
!./part1

Push success: 512 / 512
Pop  success: 512 / 512
Final top value: -1
10 popped values: 127 126 125 124 123 122 121 120 119 118 


### **2 часть. Параллельная очередь FIFO**

А здесь мне необходимо реализовать параллельную очередь FIFO на GPU с использованием глобальной памяти и атомарных операций, обеспечив корректный порядок обработки данных и безопасный параллельный доступ нескольких потоков.

In [14]:
%%writefile part2.cu

#include <cuda_runtime.h>        // для функций CUDA, которые мне надо
#include <iostream>                  // ввод/вывод
#include <cstdlib>                             // подключает exit()
using namespace std;                           // чтобы не писать std



void cuda_ok(cudaError_t err, const char* msg) {           // функция проверки ошибок CUDA
    if (err != cudaSuccess) {                                 // если произошла ошибка
        cout << "CUDA error (" << msg << "): "            // печатает место ошибки
             << cudaGetErrorString(err) << endl;              // печатает текст ошибки
        exit(1);                                // завершает программу
    }
}

struct Queue {                                 // структура очереди
    int* data;                                    // массив данных очереди в GPU памяти
    int head;                           // индекс головы, то еть откуда читаем
    int tail;                                 // индекс хвоста, то еть куда пишем
    int capacity;                                // максимальная емкость очереди

    __device__ void init(int* buffer, int size) {               // инициализация очереди на GPU
        data = buffer;                                // запоминает указатель на буфер
        head = 0;                                 // голова начинается с 0
        tail = 0;                              // хвост начинается с 0
        capacity = size;                            // сохраняет емкость
    }
    __device__ bool enqueue(int value) {                 // добавляет элемент в очередь
        int pos = atomicAdd(&tail, 1);              // атомарно берет позицию и увеличивает tail
        if (pos < capacity) {                 // если место в очереди есть
            data[pos] = value;                 // записывает значение в очередь
            return true;                          // сообщает успех
        }
        return false;                          // сообщает неуспех, очередь переполнена
    }
    __device__ bool dequeue(int* value) {                  // удаляет элемент из очереди
        int pos = atomicAdd(&head, 1);                      // атомарно берёт позицию и увеличивает head
        if (pos < tail) {                              // если элемент реально существует
            *value = data[pos];                          // записывает извлечённое значение
            return true;                           // сообщает успех
        }
        return false;                                // сообщает неуспех, очередь пуста
    }
};



__global__ void kernel_init(Queue* q, int* buffer, int cap) {           // ядро инициализации очереди
    if (threadIdx.x == 0 && blockIdx.x == 0) {                   // только один поток выполняет
        q->init(buffer, cap);                              // инициализирует очередь
    }
}
__global__ void kernel_enqueue(Queue* q, int n_enq, int* ok_enq) {       // ядро для параллельных enqueue
    int tid = threadIdx.x + blockIdx.x * blockDim.x;                  // вычисляет глобальный id потока
    if (tid < n_enq) {                                  // если поток входит в число enqueue
        bool ok = q->enqueue(tid);                             // добавляет tid в очередь
        ok_enq[tid] = ok ? 1 : 0;                               // пишет 1 если успешно
    }
}
__global__ void kernel_dequeue(Queue* q, int n_deq, int* out, int* ok_deq) {                 // ядро для параллельных dequeue
    int tid = threadIdx.x + blockIdx.x * blockDim.x;                             // вычисляет глобальный id потока
    if (tid < n_deq) {                                                             // если поток входит в число dequeue
        int val = -1;                                              // значение по умолчанию
        bool ok = q->dequeue(&val);                                     // пытается достать значение
        out[tid] = val;                                              // сохраняет значение
        ok_deq[tid] = ok ? 1 : 0;                                    // пишет 1 если успешно
    }
}
__global__ void kernel_get_head_tail(Queue* q, int* out_head, int* out_tail) {           // ядро чтения head и tail
    if (threadIdx.x == 0 && blockIdx.x == 0) {                            // только один поток выполняет
        out_head[0] = q->head;                                          // копирует head
        out_tail[0] = q->tail;                                            // копирует tail
    }
}




int main() {
    const int CAP = 1024;                          // емкость очереди
    const int N_ENQ = 512;                              // сколько добавляем
    const int N_DEQ = 512;                       // сколько извлекаем
    int* d_buffer = nullptr;                           // буфер очереди на GPU
    Queue* d_queue = nullptr;                   // объект очереди на GPU
    int* d_ok_enq = nullptr;                   // массив успешности enqueue на GPU
    int* d_ok_deq = nullptr;                     // массив успешности dequeue на GPU
    int* d_out = nullptr;                 // массив извлеченных значений на GPU
    int* d_head_val = nullptr;                            // массив для head на GPU
    int* d_tail_val = nullptr;                          // массив для tail на GPU

    cuda_ok(cudaMalloc((void**)&d_buffer, CAP * (int)sizeof(int)), "cudaMalloc d_buffer");               // выделяет буфер очереди
    cuda_ok(cudaMalloc((void**)&d_queue, (int)sizeof(Queue)), "cudaMalloc d_queue");                  // выделяет память под Queue
    cuda_ok(cudaMalloc((void**)&d_ok_enq, N_ENQ * (int)sizeof(int)), "cudaMalloc d_ok_enq");                 // выделяет ok_enq
    cuda_ok(cudaMalloc((void**)&d_ok_deq, N_DEQ * (int)sizeof(int)), "cudaMalloc d_ok_deq");                       // выделяет ok_deq
    cuda_ok(cudaMalloc((void**)&d_out, N_DEQ * (int)sizeof(int)), "cudaMalloc d_out");                         // выделяет out
    cuda_ok(cudaMalloc((void**)&d_head_val, (int)sizeof(int)), "cudaMalloc d_head_val");                          // выделяет head_val
    cuda_ok(cudaMalloc((void**)&d_tail_val, (int)sizeof(int)), "cudaMalloc d_tail_val");                         // выделяет tail_val

    kernel_init<<<1, 1>>>(d_queue, d_buffer, CAP);                            // запускает инициализацию очереди
    cuda_ok(cudaGetLastError(), "kernel_init launch");                           // проверяет запуск
    cuda_ok(cudaDeviceSynchronize(), "kernel_init sync");                // ждет завершения
    int blockSize = 256;                                               // размер блока
    int gridEnq = (N_ENQ + blockSize - 1) / blockSize;                       // количество блоков для enqueue
    int gridDeq = (N_DEQ + blockSize - 1) / blockSize;                             // количество блоков для dequeue
    kernel_enqueue<<<gridEnq, blockSize>>>(d_queue, N_ENQ, d_ok_enq);                         // параллельно добавляет значения
    cuda_ok(cudaGetLastError(), "kernel_enqueue launch");                                          // проверяет запуск
    cuda_ok(cudaDeviceSynchronize(), "kernel_enqueue sync");                                      // ждет завершения
    kernel_dequeue<<<gridDeq, blockSize>>>(d_queue, N_DEQ, d_out, d_ok_deq);                        // параллельно извлекает значения
    cuda_ok(cudaGetLastError(), "kernel_dequeue launch");                                      // проверяет запуск
    cuda_ok(cudaDeviceSynchronize(), "kernel_dequeue sync");                                     // ждет завершения
    kernel_get_head_tail<<<1, 1>>>(d_queue, d_head_val, d_tail_val);                          // читает head и tail
    cuda_ok(cudaGetLastError(), "kernel_get_head_tail launch");                                // проверяет запуск
    cuda_ok(cudaDeviceSynchronize(), "kernel_get_head_tail sync");                         // ждет завершения

    int* h_ok_enq = new int[N_ENQ];                   // ok_enq на CPU
    int* h_ok_deq = new int[N_DEQ];               // ok_deq на CPU
    int* h_out = new int[N_DEQ];                    // out на CPU
    int h_head = 0;                            // head на CPU
    int h_tail = 0;                               // tail на CPU
    cuda_ok(cudaMemcpy(h_ok_enq, d_ok_enq, N_ENQ * (int)sizeof(int), cudaMemcpyDeviceToHost), "copy ok_enq");                 // копирует ok_enq
    cuda_ok(cudaMemcpy(h_ok_deq, d_ok_deq, N_DEQ * (int)sizeof(int), cudaMemcpyDeviceToHost), "copy ok_deq");                 // копирует ok_deq
    cuda_ok(cudaMemcpy(h_out, d_out, N_DEQ * (int)sizeof(int), cudaMemcpyDeviceToHost), "copy out");                     // копирует out
    cuda_ok(cudaMemcpy(&h_head, d_head_val, (int)sizeof(int), cudaMemcpyDeviceToHost), "copy head");                       // копирует head
    cuda_ok(cudaMemcpy(&h_tail, d_tail_val, (int)sizeof(int), cudaMemcpyDeviceToHost), "copy tail");                   // копирует tail

    int enq_success = 0;                           // счетчик успешных enqueue
    for (int i = 0; i < N_ENQ; i++) {                  // цикл по enqueue
        enq_success += h_ok_enq[i];                // суммирует успехи enqueue
    }
    int deq_success = 0;                                // счетчик успешных dequeue
    for (int i = 0; i < N_DEQ; i++) {               // цикл по dequeue
        deq_success += h_ok_deq[i];                   // суммирует успехи dequeue
    }

    cout << "enqueue success: " << enq_success << " / " << N_ENQ << endl;                    // печатает успех enqueue
    cout << "dequeue success: " << deq_success << " / " << N_DEQ << endl;                       // печатает успех dequeue
    cout << "final head: " << h_head << ", final tail: " << h_tail << endl;                    // печатает head и tail
    cout << "first 10 dequeued values: ";
    for (int i = 0; i < 10 && i < N_DEQ; i++) {                                     // выводит первые 10 значений
        cout << h_out[i] << " ";
    }
    cout << endl;

    cuda_ok(cudaFree(d_buffer), "cudaFree d_buffer");           // освобождает буфер
    cuda_ok(cudaFree(d_queue), "cudaFree d_queue");                   // освобождает объект очереди
    cuda_ok(cudaFree(d_ok_enq), "cudaFree d_ok_enq");                      // освобождает ok_enq
    cuda_ok(cudaFree(d_ok_deq), "cudaFree d_ok_deq");                       // освобождает ok_deq
    cuda_ok(cudaFree(d_out), "cudaFree d_out");                         // освобождает out
    cuda_ok(cudaFree(d_head_val), "cudaFree d_head_val");                 // освобождает head_val
    cuda_ok(cudaFree(d_tail_val), "cudaFree d_tail_val");                  // освобождает tail_val

    delete[] h_ok_enq;                            // освобождает ok_enq на CPU
    delete[] h_ok_deq;                             // освобождает ok_deq на CPU
    delete[] h_out;                         // освобождает out на CPU

    return 0;        // завершает программу
}

Overwriting part2.cu


In [15]:
!nvcc part2.cu -o part2 -arch=compute_75 -code=sm_75

In [16]:
!./part2

enqueue success: 512 / 512
dequeue success: 512 / 512
final head: 512, final tail: 512
first 10 dequeued values: 256 257 258 259 260 261 262 263 264 265 


### **3 часть. Сравнение параллельного стека и очереди на GPU**

В данной части проводится сравнение двух параллельных структур данных, реализованных на GPU: стека LIFO и очереди FIFO. Обе структуры используют глобальную память и атомарные операции для обеспечения корректного параллельного доступа нескольких потоков.

**Сравнение принципов работы**

Стек работает по принципу последним пришел, первым вышел. Все операции push и pop синхронизируются через атомарные операции над одной переменной top.

Очередь же работает по принципу первым пришёл, первым вышел. Для этого используются две атомарные переменные head и tail, что делает логику более сложной по сравнению со стеком.

**Корректность работы**

В обоих случаях все операции добавления и извлечения элементов были выполнены успешно. Финальные значения управляющих индексов (top для стека, head и tail для очереди) подтверждают, что структуры данных находятся в корректном состоянии после завершения параллельных операций.

**Производительность и сложность**

Реализация стека является более простой и требует меньшего количества управляющих переменных. Очередь, в свою очередь, сложнее в реализации, так как требует синхронизации двух индексов. При большом количестве потоков обе структуры могут испытывать снижение производительности из-за частого использования атомарных операций.

**Вывод**

В результате сравнения можно сделать вывод, что стек проще в реализации и удобен для задач, где не важен порядок обработки данных. Очередь подходит для сценариев, где необходимо строго соблюдать порядок обработки элементов. Оба подхода демонстрируют основные принципы синхронизации и параллельного доступа к данным на GPU.